In [ ]:
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import spacy

In [ ]:
df=pd.read_csv("../data/newcorp.csv")

In [ ]:
df.info()

In [ ]:
text_list=[]
label_list=[]

In [ ]:
def datasetmaker(x):
    x=json.loads(x)
    for i in range(0,len(x["paragraphs"])):
        for j in range(0,len(x["paragraphs"][i])):
            text_list.append(x["paragraphs"][i][j]["text"])
            label_list.append(x["paragraphs"][i][j]["label"])

In [ ]:
df["article_segments"].apply(lambda x:datasetmaker(x))

In [ ]:
len(text_list)

In [ ]:
len(label_list)

In [ ]:
df_main={"text":text_list,"label":label_list}

In [ ]:
df_main=pd.DataFrame(df_main)

In [ ]:
df_main.head()

In [ ]:
len(text_list)

In [ ]:
df_main["label"].value_counts()

In [ ]:
sns.countplot(data=df_main,x="label")
plt.xticks(rotation=45, ha='right')

In [ ]:
df_main[df_main["label"]=="no-unit"]["text"]

In [ ]:
len_no_unit=df_main[df_main["label"]=="no-unit"]["text"].apply(lambda x:len(x.split(" ")))

In [ ]:
len_no_unit.value_counts().head()

#### Feel that a lot of no unit labels consist of just a single punctuation
#### Discarding these should be considered 
#### Makes the dataset very imbalanced 

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df_main["label"]=le.fit_transform(df_main["label"])

In [ ]:
X=df_main["text"]
y=df_main["label"]

In [ ]:
y.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.20, random_state=42,stratify=y)

In [ ]:
Train=pd.concat([X_train,y_train],axis=1)

In [ ]:
Test=pd.concat([X_test,y_test],axis=1)

In [ ]:
Train.to_csv("train.csv",index=False)
Test.to_csv("test.csv",index=False)

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset('csv', data_files={'train': ['train.csv'],'test': 'test.csv'})

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
dataset

In [ ]:
def tokenize_data(example):
    return tokenizer(example['text'], padding='max_length',truncation=True)

In [ ]:
tokenized_datasets = dataset.map(tokenize_data, batched=True)

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(5000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
tokenized_datasets

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=8)

In [ ]:
from transformers import TrainingArguments,Trainer

training_args = TrainingArguments("test_trainer", num_train_epochs=3)

In [ ]:

from datasets import load_metric

metric = load_metric("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()